# 🐞 Debug Dashboard: Bayesian-AI System

**Objective:** Rapid system validation and troubleshooting.
**Status:** 🟢 Pass | 🔴 Fail | 🟡 Warning

## 1. Environment Check 🖥️
Verify dependencies, CUDA availability, and data presence.

In [ ]:
import sys
import os
import glob
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output
from numba import cuda
from pathlib import Path

# Dynamic Root Finding
current_dir = Path(os.getcwd())
project_root = None

# Look for AGENTS.md as a marker for the root
for parent in [current_dir] + list(current_dir.parents):
    if (parent / 'AGENTS.md').exists():
        project_root = parent
        break

if project_root:
    sys.path.append(str(project_root))
    print(f"Project Root Found: {project_root}")
else:
    # Fallback to current dir if not found (though less likely to work)
    project_root = current_dir
    sys.path.append(str(project_root))
    print("Project Root Not Found! Using current directory.")

print(f"Python Version: {sys.version.split()[0]}")

# Check CUDA
try:
    if cuda.is_available():
        print(f"🟢 CUDA Available: {cuda.detect()}")
    else:
        print("🟡 CUDA Not Available (using CPU)")
except Exception as e:
    print(f"🔴 CUDA Check Failed: {e}")

# Check Data
data_path = project_root / 'DATA' / 'RAW' / '*'
raw_files = glob.glob(str(data_path))
if raw_files:
    print(f"🟢 Data Files Found ({len(raw_files)}): {', '.join([os.path.basename(f) for f in raw_files])}")
else:
    print(f"🔴 No files in {data_path}")

## 2. Data Pipeline Test 📊
Load a sample file and visualize to ensure data integrity.

In [ ]:
from training.orchestrator import get_data_source

try:
    if not raw_files:
        raise FileNotFoundError("No data files to test")
        
    target_file = raw_files[0]
    print(f"Loading {target_file}...")
    
    df = get_data_source(target_file)
    
    print(f"🟢 Load Success! Rows: {len(df)}")
    print(f"Columns: {list(df.columns)}")
    display(df.head())
    
    # Simple Plot
    plot_df = df.head(1000)
    fig = go.Figure(data=[go.Candlestick(x=plot_df.index if 'timestamp' not in plot_df.columns else plot_df['timestamp'],
                open=plot_df['open'] if 'open' in plot_df.columns else plot_df['price'],
                high=plot_df['high'] if 'high' in plot_df.columns else plot_df['price'],
                low=plot_df['low'] if 'low' in plot_df.columns else plot_df['price'],
                close=plot_df['close'] if 'close' in plot_df.columns else plot_df['price'])])
    fig.update_layout(title=f'Price Sample: {os.path.basename(target_file)}', xaxis_rangeslider_visible=False)
    fig.show()
    
except Exception as e:
    print(f"🔴 Data Pipeline Failed: {e}")

## 3. Core Component Tests ⚙️
Isolated tests for key modules.

In [ ]:
from core.state_vector import StateVector
from core.bayesian_brain import BayesianBrain
from core.layer_engine import LayerEngine
from cuda.velocity_gate import CUDAVelocityGate
import time

print("--- Component Status ---")

# 1. StateVector
try:
    sv = StateVector.null_state()
    assert hash(sv) is not None
    print("🟢 StateVector: Operational")
except Exception as e:
    print(f"🔴 StateVector: Failed ({e})")

# 2. BayesianBrain
try:
    bb = BayesianBrain()
    prob_table_path = project_root / 'probability_table.pkl'
    if prob_table_path.exists():
        bb.load(str(prob_table_path))
        print(f"🟢 BayesianBrain: Loaded {len(bb.table)} states from disk")
    else:
        print("🟡 BayesianBrain: No existing table found (Clean Start)")
except Exception as e:
    print(f"🔴 BayesianBrain: Failed ({e})")

# 3. LayerEngine
try:
    le = LayerEngine(use_gpu=False) # Test CPU init first
    print("🟢 LayerEngine: Initialized")
except Exception as e:
    print(f"🔴 LayerEngine: Failed ({e})")

# 4. CUDA VelocityGate
try:
    vg = CUDAVelocityGate(use_gpu=True)
    print(f"🟢 VelocityGate: Initialized (GPU={vg.use_gpu})")
except Exception as e:
    print(f"🔴 VelocityGate: Failed ({e})")

## 4. Mini Training Run 🏃
Execute `orchestrator.py` in a subprocess with live output.

In [ ]:
import subprocess
import threading

def run_training_process(iterations=5):
    output_dir = project_root / 'debug_outputs'
    output_dir.mkdir(exist_ok=True)
    
    cmd = [
        "python", "training/orchestrator.py",
        "--data-dir", "DATA/RAW",
        "--iterations", str(iterations),
        "--output", "debug_outputs/"
    ]
    
    print(f"Executing: {' '.join(cmd)}")
    print(f"CWD: {project_root}")
    
    process = subprocess.Popen(
        cmd,
        cwd=str(project_root),
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )
    
    return process

btn_run = widgets.Button(description="Start Mini Run (5 Iters)", button_style='success')
out_log = widgets.Output(layout={'border': '1px solid black', 'height': '300px', 'overflow_y': 'scroll'})

def on_run_click(b):
    out_log.clear_output()
    with out_log:
        print("Starting subprocess...")
        proc = run_training_process()
        
        while True:
            line = proc.stdout.readline()
            if not line and proc.poll() is not None:
                break
            if line:
                print(line.strip())
        
        if proc.returncode == 0:
            print("\n✅ Process Completed Successfully")
        else:
            print(f"\n❌ Process Failed with Code {proc.returncode}")

btn_run.on_click(on_run_click)
display(btn_run, out_log)

## 5. Probability Table Analysis 📈
Analyze the learned states from the run.

In [ ]:
import pickle

def analyze_table(filepath):
    if not os.path.exists(filepath):
        print(f"🔴 File not found: {filepath}")
        return
        
    with open(filepath, 'rb') as f:
        data = pickle.load(f)
    
    table = data['table']
    print(f"Loaded {len(table)} states.")
    
    # Convert to DataFrame for analysis
    records = []
    for state, stats in table.items():
        total = stats['total']
        wins = stats['wins']
        if total > 0:
            records.append({
                'total': total,
                'wins': wins,
                'win_rate': wins/total,
                'L1': state.L1_bias,
                'L5': state.L5_trend
            })
            
    if not records:
        print("No records to analyze.")
        return
        
    df_stats = pd.DataFrame(records)
    
    # Histogram of sample sizes
    fig1 = px.histogram(df_stats, x='total', nbins=50, title='Distribution of Samples per State')
    fig1.show()
    
    # Win Rate vs Sample Size
    fig2 = px.scatter(df_stats, x='total', y='win_rate', title='Win Rate vs Sample Size',
                      hover_data=['L1', 'L5'])
    fig2.show()
    
    # High Confidence States
    high_conf = df_stats[df_stats['total'] > 10].sort_values('win_rate', ascending=False).head(10)
    print("Top 10 High Confidence States:")
    display(high_conf)

# Check debug output first, then main
debug_prob_path = project_root / 'debug_outputs' / 'probability_table.pkl'
main_prob_path = project_root / 'probability_table.pkl'

if debug_prob_path.exists():
    analyze_table(str(debug_prob_path))
elif main_prob_path.exists():
    analyze_table(str(main_prob_path))
else:
    print("🟡 No probability table found. Run training first.")

## 6. Performance Profiling ⏱️
Benchmark key operations.

In [ ]:
import timeit

print("Benchmarking StateVector Hashing...")
setup_code = "from core.state_vector import StateVector; sv = StateVector.null_state()"
t = timeit.timeit("hash(sv)", setup=setup_code, number=100000)
print(f"Create & Hash 100k states: {t:.4f}s")

print("\nBenchmarking VelocityGate (CPU fallback check)...")
setup_vg = """
from cuda.velocity_gate import CUDAVelocityGate
import numpy as np
vg = CUDAVelocityGate(use_gpu=False)
prices = np.random.random(100).astype(np.float32)
"""
t_vg = timeit.timeit("vg.detect_cascade(prices)", setup=setup_vg, number=1000)
print(f"1000 detections (CPU): {t_vg:.4f}s")

## 7. DOE Simulation Preview 🧪
Preview parameter combinations for optimization.

In [ ]:
import itertools

param_grid = {
    'min_prob': [0.70, 0.75, 0.80, 0.85],
    'min_conf': [0.20, 0.30, 0.40],
    'stop_loss': [10, 20, 30],
    'kill_zones': ['tight', 'wide']
}

keys, values = zip(*param_grid.items())
combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

print(f"Total Combinations: {len(combinations)}")
print("Sample First 5:")
for c in combinations[:5]:
    print(c)

print("\nNOTE: Full DOE integration requires implementing the grid search loop in orchestrator.")

## 8. Quick Fixes & Utilities 🛠️

In [ ]:
import shutil

def clean_pycache():
    print("Cleaning __pycache__...")
    count = 0
    # Walk the project root
    for root, dirs, files in os.walk(str(project_root)):
        for d in dirs:
            if d == '__pycache__':
                shutil.rmtree(os.path.join(root, d))
                count += 1
    print(f"Removed {count} directories.")

btn_clean = widgets.Button(description="Clear PyCache")
btn_clean.on_click(lambda b: clean_pycache())
display(btn_clean)